# AI Lesson 04b: GROUP BY for Machine Learning

**Course:** Applications of Artificial Intelligence  
**Focus:** Grouping and Aggregating Data for ML  

---

## Learning Objectives

By the end of this short lesson, you will:
- Understand what `GROUP BY` does and why it's crucial for ML
- Calculate statistics per category (team, season, etc.)
- Use `HAVING` to filter grouped results
- Prepare aggregated datasets for machine learning

**Why this matters for ML:**  
In our next lesson (ai05), we'll build a **Random Forest model** to predict NBA game winners. To do that, we need to calculate **team season averages** using the exact GROUP BY skills you'll learn today.

---

## Setup

In [2]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('nba_5seasons.db')
print("✅ Connected to NBA database")

✅ Connected to NBA database


---

## Part 1: The Problem GROUP BY Solves

### Without GROUP BY

Let's say we want to know the **average points per game for each team** in the 2024-25 season.

**Without GROUP BY**, we'd have to:
1. Filter for each team individually
2. Calculate the average
3. Repeat 30 times (once per team!)

That's tedious and inefficient. Let's see this problem:

In [3]:
# The hard way: Calculate average points for ONE team
query = """
SELECT AVG(pts) as avg_points
FROM team_game_stats
WHERE season = '2024-25' 
  AND team_id = 1610612739
"""

result = pd.read_sql(query, conn)
print("Average points for team 1610612739 (Cavaliers):")
display(result)

print("\n❌ Problem: We'd have to write 30 different queries for all teams!")

Average points for team 1610612739 (Cavaliers):


,avg_points
0,121.939024



❌ Problem: We'd have to write 30 different queries for all teams!


### With GROUP BY 

**GROUP BY** splits our data into groups (one per team) and calculates statistics for each group **automatically**.

Think of it like this:
```
All games → Split by team_id → Calculate AVG(pts) for each team → Done!
```

In [4]:
# The smart way: Calculate average points for ALL teams at once
query = """
SELECT 
    team_id,
    AVG(pts) as avg_points
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY avg_points DESC
"""

result = pd.read_sql(query, conn)
print("✅ Average points for ALL teams in one query!")
display(result.head(10))

✅ Average points for ALL teams in one query!


,team_id,avg_points
0,1610612739,121.939024
1,1610612763,121.707317
2,1610612743,120.756098
3,1610612760,120.500000
4,1610612737,118.182927
5,1610612741,117.804878
6,1610612754,117.365854
7,1610612738,116.268293
8,1610612752,115.780488
9,1610612758,115.731707


### Key Insight

**GROUP BY pattern:**
```sql
SELECT 
    category_column,           -- What to group by
    AGGREGATE_FUNCTION(value)  -- What to calculate
FROM table
GROUP BY category_column
```

**Rules:**
1. Columns in SELECT must EITHER be:
   - In the GROUP BY clause, OR
   - Inside an aggregate function (AVG, SUM, COUNT, MAX, MIN)
2. GROUP BY creates one row per unique value in the grouping column

---

## Part 2: Practice GROUP BY

### Task 1: Count Games Per Season

Write a query to count how many games were played in EACH season.

**Hints:**
- Use `COUNT(*)` to count rows
- Group by the `season` column
- Order by season

In [5]:
query = """
SELECT season, COUNT(*) as game_count
FROM team_game_stats
GROUP BY season
ORDER BY season
"""

result = pd.read_sql(query, conn)
display(result)

,season,game_count
0,2021-22,2460
1,2022-23,2460
2,2023-24,2460
3,2024-25,2460
4,2025-26,1002


**Expected:** One row per season with game count

### Task 2: Team Season Averages

For the 2024-25 season, calculate average points, assists, and rebounds for EACH team.

**Hints:**
- Filter with WHERE before grouping
- Use AVG() for each stat
- Order by avg_points descending

In [6]:
query = """
SELECT 
    team_id,
    AVG(pts) as avg_points,
    AVG(ast) as avg_assists,
    AVG(reb) as avg_rebounds
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY avg_points DESC
"""

result = pd.read_sql(query, conn)
display(result.head(10))

,team_id,avg_points,avg_assists,avg_rebounds
0,1610612739,121.939024,28.097561,45.402439
1,1610612763,121.707317,28.414634,47.256098
2,1610612743,120.756098,31.000000,45.743902
3,1610612760,120.500000,26.853659,44.780488
4,1610612737,118.182927,29.585366,44.500000
5,1610612741,117.804878,29.060976,45.853659
6,1610612754,117.365854,29.243902,41.829268
7,1610612738,116.268293,26.146341,45.317073
8,1610612752,115.780488,27.487805,42.646341
9,1610612758,115.731707,26.451220,44.195122


### Task 3: Count Wins Per Team

For the 2024-25 season, count how many wins each team has.

**Hints:**
- WHERE season = '2024-25' AND wl = 'W'
- COUNT(*) will count wins
- GROUP BY team_id
- ORDER BY wins descending

In [7]:
query = """
SELECT 
    team_id,
    COUNT(*) as wins
FROM team_game_stats
WHERE season = '2024-25' AND wl = 'W'
GROUP BY team_id
ORDER BY wins DESC
"""

result = pd.read_sql(query, conn)
display(result.head(10))

,team_id,wins
0,1610612760,68
1,1610612739,64
2,1610612738,61
3,1610612745,52
4,1610612752,51
5,1610612754,50
6,1610612747,50
7,1610612746,50
8,1610612743,50
9,1610612750,49


---

## Part 3: Multiple Aggregations

### Task 4: Complete Team Stats

For each team in 2024-25, calculate:
- Game count
- Average points
- Highest score (MAX)
- Lowest score (MIN)

**All in one query!**

In [8]:
query = """
SELECT 
    team_id,
    COUNT(*) as game_count,
    AVG(pts) as avg_points,
    MAX(pts) as highest_score,
    MIN(pts) as lowest_score
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY avg_points DESC
"""

result = pd.read_sql(query, conn)
display(result.head(10))

,team_id,game_count,avg_points,highest_score,lowest_score
0,1610612739,82,121.939024,149,93
1,1610612763,82,121.707317,155,99
2,1610612743,82,120.756098,149,87
3,1610612760,82,120.500000,146,98
4,1610612737,82,118.182927,148,92
5,1610612741,82,117.804878,146,91
6,1610612754,82,117.365854,162,83
7,1610612738,82,116.268293,142,76
8,1610612752,82,115.780488,149,94
9,1610612758,82,115.731707,141,88


**Notice:** You can use as many aggregate functions as you want in one query!

---

## Part 4: HAVING - Filter Groups

### The Problem

What if we only want teams with **50+ wins**?

**We can't use WHERE** because WHERE filters individual rows BEFORE grouping.

**We need HAVING** to filter groups AFTER aggregation.

### WHERE vs HAVING

| Clause | When | Filters | Example |
|--------|------|---------|----------|
| WHERE | Before grouping | Individual rows | `WHERE season = '2024-25'` |
| HAVING | After grouping | Aggregated groups | `HAVING COUNT(*) >= 50` |

**Query order:**
```sql
SELECT ...
FROM ...
WHERE ...      -- Filter rows first
GROUP BY ...   -- Then group
HAVING ...     -- Then filter groups
ORDER BY ...   -- Finally sort
```

### Example: Elite Teams (50+ Wins)

In [9]:
query = """
SELECT 
    team_id,
    COUNT(*) as wins
FROM team_game_stats
WHERE season = '2024-25' 
  AND wl = 'W'
GROUP BY team_id
HAVING COUNT(*) >= 50    -- Filter groups with 50+ wins
ORDER BY wins DESC
"""

result = pd.read_sql(query, conn)
print("Elite teams with 50+ wins:")
display(result)

Elite teams with 50+ wins:


,team_id,wins
0,1610612760,68
1,1610612739,64
2,1610612738,61
3,1610612745,52
4,1610612752,51
5,1610612754,50
6,1610612747,50
7,1610612746,50
8,1610612743,50


### Task 5: High-Scoring Teams

Find all teams averaging 110+ points per game in 2024-25.

**Hints:**
- GROUP BY team_id
- Use HAVING with AVG(pts)
- Include the avg_points in your SELECT

In [10]:
query = """
SELECT 
	team_id,
	COUNT(*) as total_games,
	SUM(CASE WHEN wl = 'W' THEN 1 ELSE 0 END) as wins,
	ROUND(SUM(CASE WHEN wl = 'W' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as win_percentage
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY win_percentage DESC
"""

result = pd.read_sql(query, conn)
display(result)

,team_id,total_games,wins,win_percentage
0,1610612760,82,68,82.9
1,1610612739,82,64,78.0
2,1610612738,82,61,74.4
3,1610612745,82,52,63.4
4,1610612752,82,51,62.2
5,1610612754,82,50,61.0
6,1610612747,82,50,61.0
7,1610612746,82,50,61.0
8,1610612743,82,50,61.0
9,1610612750,82,49,59.8


---

## Part 5: Preview - GROUP BY in Machine Learning

### Real Use Case: Random Forest Prep

In **ai05 (next lesson)**, we'll build a Random Forest model to predict game winners.

**The query we'll use:**

In [14]:
# This is the EXACT query from ai05 - GROUP BY in action!
team_avg_query = """
SELECT 
    team_id,
    AVG(pts) as avg_pts,
    AVG(ast) as avg_ast,
    AVG(reb) as avg_reb,
    AVG(stl) as avg_stl,
    AVG(blk) as avg_blk,
    AVG(tov) as avg_tov,
    COUNT(*) as games_played
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY avg_pts DESC
"""

team_averages = pd.read_sql(team_avg_query, conn)
print("Team Season Averages (for ML prediction):")
display(team_averages.head(10))

Team Season Averages (for ML prediction):


,team_id,avg_pts,avg_ast,avg_reb,avg_stl,avg_blk,avg_tov,games_played
0,1610612739,121.939024,28.097561,45.402439,8.170732,4.268293,13.195122,82
1,1610612763,121.707317,28.414634,47.256098,8.853659,5.634146,15.670732,82
2,1610612743,120.756098,31.000000,45.743902,8.000000,4.902439,14.256098,82
3,1610612760,120.500000,26.853659,44.780488,10.329268,5.670732,11.719512,82
4,1610612737,118.182927,29.585366,44.500000,9.731707,5.109756,15.524390,82
5,1610612741,117.804878,29.060976,45.853659,7.573171,4.707317,14.670732,82
6,1610612754,117.365854,29.243902,41.829268,8.463415,5.463415,13.195122,82
7,1610612738,116.268293,26.146341,45.317073,7.182927,5.512195,11.865854,82
8,1610612752,115.780488,27.487805,42.646341,8.182927,3.951220,13.268293,82
9,1610612758,115.731707,26.451220,44.195122,7.560976,4.353659,13.317073,82


### How This Powers ML

In ai05, we'll:
1. Pull these team averages using GROUP BY ✅ (what you just learned!)
2. Calculate differentials (Team A avg - Team B avg)
3. Feed differentials into Random Forest model
4. Predict: "Will the home team win?"

**Without GROUP BY, this wouldn't be possible!**

---

## Part 6: Challenge Tasks

### Challenge 1: Win Percentage

Calculate each team's win percentage for 2024-25.

**Formula:** `(wins / total_games) * 100`

**Hints:**
- Count total games with `COUNT(*)`
- Count wins with `SUM(CASE WHEN wl = 'W' THEN 1 ELSE 0 END)`
- Use `* 100.0` to get percentage
- Round with `ROUND(value, 1)` for one decimal place

In [19]:
query = """
SELECT
    team_id,
    COUNT(*) AS total_games,
    SUM(CASE WHEN wl = 'W' THEN 1 ELSE 0 END) AS wins,
    ROUND(
        (SUM(CASE WHEN wl = 'W' THEN 1 ELSE 0 END) * 100.0) / COUNT(*),
        1
    ) AS win_percentage
FROM team_game_stats
WHERE season = '2024-25'
GROUP BY team_id
ORDER BY win_percentage DESC;
"""

result = pd.read_sql(query, conn)
display(result.head(10))

,team_id,total_games,wins,win_percentage
0,1610612760,82,68,82.9
1,1610612739,82,64,78.0
2,1610612738,82,61,74.4
3,1610612745,82,52,63.4
4,1610612752,82,51,62.2
5,1610612754,82,50,61.0
6,1610612747,82,50,61.0
7,1610612746,82,50,61.0
8,1610612743,82,50,61.0
9,1610612750,82,49,59.8


### Challenge 2: Season-by-Season Scoring Trends

Calculate average points for EACH season to see if scoring is increasing over time.

**Hint:** GROUP BY season, ORDER BY season

In [20]:
query = """
SELECT
    season,
    ROUND(AVG(pts), 1) AS avg_points
FROM team_game_stats
GROUP BY season
ORDER BY season;
"""

result = pd.read_sql(query, conn)
display(result)

,season,avg_points
0,2021-22,110.6
1,2022-23,114.7
2,2023-24,114.2
3,2024-25,113.8
4,2025-26,116.7


**Question:** Is scoring going up or down over the years?
Up

---

## Cleanup

In [21]:
conn.close()
print("✅ Database connection closed")

✅ Database connection closed


---

## Summary

### What You Learned

✅ **GROUP BY** - Split data by category and aggregate  
✅ **Aggregate functions** - COUNT, SUM, AVG, MAX, MIN  
✅ **HAVING** - Filter groups after aggregation  
✅ **WHERE vs HAVING** - When to use each  
✅ **ML connection** - How GROUP BY prepares data for models  

### Key Patterns

```sql
-- Basic grouping
SELECT category, AVG(value)
FROM table
GROUP BY category

-- With filtering
SELECT category, COUNT(*)
FROM table
WHERE condition        -- Filter rows BEFORE grouping
GROUP BY category
HAVING COUNT(*) > 50   -- Filter groups AFTER aggregating
ORDER BY COUNT(*) DESC
```

### Complete Query Order

```sql
SELECT      -- What columns/calculations
FROM        -- Which table
WHERE       -- Filter individual rows
GROUP BY    -- Split into groups
HAVING      -- Filter groups
ORDER BY    -- Sort results
LIMIT       -- Limit results
```

---

## Next Up: AI05 - Random Forest 

You're now ready for **ai05** where you'll:
1. Pull live NBA data from an API
2. Use GROUP BY to calculate team averages (exactly what you learned today!)
3. Build a Random Forest model
4. Predict NBA game winners

**The GROUP BY skills you learned today are essential for that lesson!**